In [ ]:
!uv sync

In [1]:
# save as test_cuda.py and run: python3 test_cuda.py

import platform

print("=== Environment ===")
print("Platform:", platform.platform())
print("Python:", platform.python_version())

try:
    import torch
except ImportError as e:
    print("\nPyTorch is not installed or not in this Python environment.")
    raise SystemExit(e)

print("\n=== PyTorch / CUDA Info ===")
print("torch.__version__:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)

cuda_available = torch.cuda.is_available()
print("torch.cuda.is_available():", cuda_available)

if not cuda_available:
    print("\nCUDA is NOT available to PyTorch in this environment.")
else:
    # Number of devices
    device_count = torch.cuda.device_count()
    print("torch.cuda.device_count():", device_count)

    for i in range(device_count):
        print(f"  device {i}: {torch.cuda.get_device_name(i)}")

    # Simple tensor test on GPU
    try:
        x = torch.rand(3, 3, device="cuda")
        y = torch.rand(3, 3, device="cuda")
        z = x @ y
        print("\nSuccessfully ran a matrix multiply on CUDA.")
        print("z.device:", z.device)
    except Exception as e:
        print("\nERROR: Allocation or compute on CUDA failed:")
        print(e)

print("\n=== Test Complete ===")

=== Environment ===
Platform: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.31
Python: 3.12.11

=== PyTorch / CUDA Info ===
torch.__version__: 2.9.1+cu128
torch.version.cuda: 12.8
torch.cuda.is_available(): True
torch.cuda.device_count(): 1
  device 0: NVIDIA GeForce RTX 5080

=== PyTorch / CUDA Info ===
torch.__version__: 2.9.1+cu128
torch.version.cuda: 12.8
torch.cuda.is_available(): True
torch.cuda.device_count(): 1
  device 0: NVIDIA GeForce RTX 5080

Successfully ran a matrix multiply on CUDA.
z.device: cuda:0

=== Test Complete ===

Successfully ran a matrix multiply on CUDA.
z.device: cuda:0

=== Test Complete ===


In [1]:
import torch
import numpy as np

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())

2.9.1+cu128
12.8
91002


In [ ]:
# alternate dependencies for running on mac without CUDA (only run if previous block fails)

!pip install pydantic
!pip install PyYAML
!pip install numpy==1.26.4
!pip install torch==2.1.1 torchdata==0.7.1
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

import torch
import dgl

In [3]:
import pandas as pd
from pathlib import Path

# Path to the small transactions CSV (relative to this notebook).
DATA_PATH = Path("dataset") / "HI-Small_Trans.csv"

# Load into a DataFrame
small_trans = pd.read_csv(DATA_PATH)

# Quick summary and preview
print(f"Loaded {len(small_trans)} rows; columns: {list(small_trans.columns)}")
small_trans.head()

Loaded 5078345 rows; columns: ['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1', 'Amount Received', 'Receiving Currency', 'Amount Paid', 'Payment Currency', 'Payment Format', 'Is Laundering']


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [4]:
# Basic analysis: currencies, banks, and other summaries
# This cell is robust to different column names: it searches for currency-like and bank-like columns

# Show shape and a small sample
rows, cols = small_trans.shape
print(f"Data shape: {rows} rows x {cols} columns")
print()
print("Sample rows:")
display(small_trans.head())

# Missing values by column (top 10)
missing_by_col = small_trans.isnull().sum().sort_values(ascending=False).head(15)
print("Top missing values by column:")
print(missing_by_col.to_string())
print()

# Find likely currency column(s)
currency_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('currency','ccy','curr'))]
if currency_candidates:
    cur_col = currency_candidates[0]
    num_currencies = small_trans[cur_col].nunique(dropna=True)
    top_currencies = small_trans[cur_col].value_counts().head(10)
    print(f"Found currency column: '{cur_col}' — {num_currencies} unique values")
    print("Top currencies (by count):")
    print(top_currencies.to_string())
else:
    cur_col = None
    print("No currency-like column found.")
    print("Columns:", list(small_trans.columns))

print()
# Find likely bank-related columns
bank_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('bank','institution','bic','iban','bankid','bank_id','bankname','bank_name'))]
if bank_candidates:
    # Count unique bank identifiers across candidate columns
    unique_banks = set()
    for c in bank_candidates:
        unique_banks.update(small_trans[c].dropna().astype(str).unique())
    num_unique_banks = len(unique_banks)
    print(f"Found bank-like columns: {bank_candidates} — approx. {num_unique_banks} unique bank identifiers (aggregated)")
else:
    num_unique_banks = None
    print("No bank-like columns found.")

print()
# Other basic summaries: amount column candidates and top senders/receivers if available
amt_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('amount','amt','value'))]
if amt_candidates:
    amt_col = amt_candidates[0]
    print(f"Found amount column: {amt_col} — summary:")
    print(small_trans[amt_col].describe())
else:
    print("No amount-like column found.")

# If sender/receiver columns exist, show top participants
party_candidates = [c for c in small_trans.columns if any(k in c.lower() for k in ('sender','receiver','originator','beneficiary','from_','to_','account'))]
if party_candidates:
    print()
    print("Top participants in party-like columns:")
    for c in party_candidates:
        print(f"Column: {c}")
# Final small sample
display(small_trans.sample(min(5, len(small_trans))))

Data shape: 5078345 rows x 11 columns

Sample rows:


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


Top missing values by column:
Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0

Found currency column: 'Receiving Currency' — 15 unique values
Top currencies (by count):
Receiving Currency
US Dollar      1879341
Euro           1172017
Swiss Franc     237884
Yuan            206551
Shekel          194988
Rupee           192065
UK Pound        181255
Ruble           157361
Yen             156319
Bitcoin         148151

Found currency column: 'Receiving Currency' — 15 unique values
Top currencies (by count):
Receiving Currency
US Dollar      1879341
Euro           1172017
Swiss Franc     237884
Yuan            206551
Shekel          194988
Rupee           192065
UK Pound        181255
Ruble           157361
Yen             156319
Bitcoin         148151

Found bank-like columns: 

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
1655130,2022/09/02 17:28,10,8001CF7B0,1292,80036A840,1489.30,US Dollar,1489.30,US Dollar,Credit Card,0
3298484,2022/09/07 01:30,27103,805AD5F90,210575,805AD5EF0,2490.17,Euro,2490.17,Euro,ACH,0
4240175,2022/09/09 00:09,138395,80E511880,239453,80ED69AB0,131181.03,Swiss Franc,131181.03,Swiss Franc,Wire,0
4886120,2022/09/10 00:00,112,80DC17D20,240243,80FC62B70,66.19,Swiss Franc,66.19,Swiss Franc,Credit Card,0
1033718,2022/09/01 21:58,16224,8066AE4F0,16788,806FE9650,189.86,US Dollar,189.86,US Dollar,Cash,0


In [5]:
# convert hex account numbers to int
hex_to_int = np.vectorize(lambda x: int(x, 16))

# create adjacency lists to represent the graph
source = hex_to_int(small_trans['Account'])
target = hex_to_int(small_trans['Account.1'])

In [11]:
from torch_geometric.data import Data
import torch
import numpy as np

# Map account IDs to a compact 0..N-1 index space to avoid huge sparse IDs
# Concatenate unique accounts from source/target and factorize
all_accounts = np.concatenate([source, target])
unique_accounts, inverse_idx = np.unique(all_accounts, return_inverse=True)
num_nodes = unique_accounts.shape[0]
# Rebuild source/target as compact indices
source_idx = inverse_idx[:source.shape[0]]
target_idx = inverse_idx[source.shape[0]:]

# Build edge_index
edge_index = torch.tensor(np.vstack([source_idx, target_idx]), dtype=torch.long)

# Create Data object
data = Data(edge_index=edge_index, num_nodes=num_nodes)
print('num_nodes:', num_nodes, 'num_edges:', edge_index.size(1))
print(data)

num_nodes: 515080 num_edges: 5078345
Data(edge_index=[2, 5078345], num_nodes=515080)


In [12]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from torch_geometric.data import Data
import torch

# extract individual edge features
time = pd.to_datetime(small_trans['Timestamp']).astype('int64') / 1e9
amount_paid = small_trans['Amount Paid'].to_numpy()
amount_received = small_trans['Amount Received'].to_numpy()

# use one-hot encoding for categorical variables
paid_enc = OneHotEncoder(sparse_output=False)
paid_currency = paid_enc.fit_transform(small_trans['Payment Currency'].to_numpy().reshape(-1, 1))

received_enc = OneHotEncoder(sparse_output=False)
received_currency = received_enc.fit_transform(small_trans['Receiving Currency'].to_numpy().reshape(-1, 1))

format_enc = OneHotEncoder(sparse_output=False)
pay_format = format_enc.fit_transform(small_trans['Payment Format'].to_numpy().reshape(-1, 1))

# combine edge features into single tensor
numeric_features = np.column_stack([time, amount_paid, amount_received])
edge_features = torch.from_numpy(np.concatenate([numeric_features, paid_currency, received_currency, pay_format], axis=1)).float()

# create edge labels
fraud_label = torch.tensor(small_trans['Is Laundering'].to_numpy(), dtype=torch.long)

# attach features and labels to PyG Data
data.edge_attr = edge_features
data.edge_label = fraud_label
print(data)

Data(edge_index=[2, 5078345], num_nodes=515080, edge_attr=[5078345, 40], edge_label=[5078345])


In [13]:
# chronological 60/20/20 split by edge index order
num_edges = data.edge_index.size(1)
train_end = int(0.6 * num_edges)
val_end = int(0.8 * num_edges)

train_mask = torch.zeros(num_edges, dtype=torch.bool)
val_mask = torch.zeros(num_edges, dtype=torch.bool)
test_mask = torch.zeros(num_edges, dtype=torch.bool)
train_mask[:train_end] = True
val_mask[train_end:val_end] = True
test_mask[val_end:] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
print('Masks set:', train_mask.sum().item(), val_mask.sum().item(), test_mask.sum().item())

Masks set: 3047007 1015669 1015669


In [ ]:
# PyG GNN model and edge classification training
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Ensure data object exists with edge_index, edge_attr, edge_label, and masks
assert data is not None, 'PyG Data not constructed yet'
num_nodes = data.num_nodes
num_edges = data.edge_index.size(1)
edge_feat_dim = data.edge_attr.size(1)

# Create simple node features if none exist (e.g., degree or identity)
if getattr(data, 'x', None) is None:
    deg = torch.zeros((num_nodes, 1), dtype=torch.float)
    deg.scatter_add_(0, data.edge_index[0].view(-1,1), torch.ones((num_edges,1)))
    data.x = deg  # use degree as a simple node feature

class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class EdgeClassifier(nn.Module):
    def __init__(self, node_hidden, edge_feat_dim, hidden=64, num_classes=2):
        super().__init__()
        # combine node embeddings of u and v with edge features
        self.mlp = nn.Sequential(
            nn.Linear(node_hidden*2 + edge_feat_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x, edge_index, edge_attr):
        u, v = edge_index
        h = torch.cat([x[u], x[v], edge_attr], dim=1)
        return self.mlp(h)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = nn.Sequential()  # placeholder to hold submodules
gnn = GNN(in_channels=data.x.size(1), hidden_channels=64).to(device)
clf = EdgeClassifier(node_hidden=64, edge_feat_dim=edge_feat_dim, hidden=128, num_classes=int(data.edge_label.max().item()+1)).to(device)

params = list(gnn.parameters()) + list(clf.parameters())
optimizer = torch.optim.Adam(params, lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Move tensors to device
data = data.to(device)
train_idx = data.train_mask.nonzero(as_tuple=False).view(-1)
val_idx = data.val_mask.nonzero(as_tuple=False).view(-1)
test_idx = data.test_mask.nonzero(as_tuple=False).view(-1)

def evaluate(split_idx):
    gnn.eval(); clf.eval()
    with torch.no_grad():
        x = gnn(data.x, data.edge_index)
        logits = clf(x, data.edge_index, data.edge_attr)
        y = data.edge_label
        preds = logits.argmax(dim=1)
        correct = (preds[split_idx] == y[split_idx]).sum().item()
        total = split_idx.numel()
        return correct / max(total, 1)

epochs = 20
for epoch in range(1, epochs+1):
    gnn.train(); clf.train()
    optimizer.zero_grad()
    x = gnn(data.x, data.edge_index)
    logits = clf(x, data.edge_index, data.edge_attr)
    loss = criterion(logits[train_idx], data.edge_label[train_idx])
    loss.backward()
    optimizer.step()
    if epoch % 2 == 0 or epoch == 1:
        train_acc = evaluate(train_idx)
        val_acc = evaluate(val_idx)
        print(f'Epoch {epoch:02d} | loss {loss.item():.4f} | train_acc {train_acc:.3f} | val_acc {val_acc:.3f}')

test_acc = evaluate(test_idx)
print('Test accuracy:', round(test_acc, 3))

Epoch 01 | loss 9361119.0000 | train_acc 0.999 | val_acc 0.999
Epoch 02 | loss 6774.2803 | train_acc 0.999 | val_acc 0.999
Epoch 02 | loss 6774.2803 | train_acc 0.999 | val_acc 0.999
Epoch 04 | loss 21247.0078 | train_acc 0.999 | val_acc 0.999
Epoch 04 | loss 21247.0078 | train_acc 0.999 | val_acc 0.999
Epoch 06 | loss 30554.0254 | train_acc 0.999 | val_acc 0.999
Epoch 06 | loss 30554.0254 | train_acc 0.999 | val_acc 0.999
Epoch 08 | loss 37014.7617 | train_acc 0.999 | val_acc 0.999
Epoch 08 | loss 37014.7617 | train_acc 0.999 | val_acc 0.999
Epoch 10 | loss 41773.3672 | train_acc 0.999 | val_acc 0.999
Epoch 10 | loss 41773.3672 | train_acc 0.999 | val_acc 0.999
Epoch 12 | loss 45290.9648 | train_acc 0.999 | val_acc 0.999
Epoch 12 | loss 45290.9648 | train_acc 0.999 | val_acc 0.999
